Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Inference PyTorch GPT2 Model with ONNX Runtime on CPU

In this tutorial, you'll be introduced to how to load a GPT2 model from PyTorch, convert it to ONNX, and inference it using ONNX Runtime.

**Note: this work is still in progresss. Need install ort_nightly package before onnxruntime 1.3.0 is ready. The performance number of ort_nightly does not reflect the final result for onnxruntime 1.3.0. **

## Prerequisites ##

If you have Jupyter Notebook, you may directly run this notebook. We will use pip to install or upgrade [PyTorch](https://pytorch.org/), [OnnxRuntime](https://microsoft.github.io/onnxruntime/) and other required packages.

Otherwise, you can setup a new environment. First, we install [AnaConda](https://www.anaconda.com/distribution/). Then open an AnaConda prompt window and run the following commands:

```console
conda create -n cpu_env python=3.6
conda activate cpu_env

conda install pytorch torchvision cpuonly -c pytorch
pip install onnxruntime
pip install transformers==2.5.1
pip install onnx psutil pytz pandas py-cpuinfo py3nvml netron

conda install jupyter
jupyter notebook
```
The last command will launch Jupyter Notebook and we can open this notebook in browser to continue.

In [ ]:
# Enable pass state in input.
enable_past_input = False

In [ ]:
import os

cache_dir = "./gpt2"
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

output_dir = './gpt2_onnx'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

## Benchmark ##

You will need git clone the onnxruntime repository like
```console
git clone https://github.com/microsoft/onnxruntime.git
```
Then update the bert_tools_dir according to the path in your machine.

In [ ]:
# Assume you have git clone the repository of onnxruntime from github.
bert_tools_dir = r'D:\Git\onnxruntime\onnxruntime\python\tools\bert'
benchmark_script = os.path.join(bert_tools_dir, 'benchmark_gpt2.py')

if enable_past_input:
    %run $benchmark_script --model_type gpt2 --cache_dir $cache_dir --output_dir $output_dir --enable_optimization --enable_past_input
else:
    %run $benchmark_script --model_type gpt2 --cache_dir $cache_dir --output_dir $output_dir --enable_optimization

If you only need the benchmark results. You can skip the remaining parts.

In the following, we will introduce the benchmark script.

### Load pretrained model

In [ ]:
from transformers import GPT2Model, GPT2Tokenizer
model_class, tokenizer_class,  model_name_or_path = (GPT2Model,  GPT2Tokenizer,  'gpt2')
tokenizer = tokenizer_class.from_pretrained(model_name_or_path, cache_dir=cache_dir)
model = model_class.from_pretrained(model_name_or_path, cache_dir=cache_dir)
model.eval().cpu()

In [ ]:
import numpy
import time

def pytorch_inference(model, input_ids, past=None, total_runs = 100):
    latency = []
    with torch.no_grad():
        for _ in range(total_runs):
            start = time.time()
            outputs = model(input_ids=input_ids, past=past)
            latency.append(time.time() - start)
            
    if total_runs > 1:
        print("PyTorch Inference time = {} ms".format(format(sum(latency) * 1000 / len(latency), '.2f')))
    
    return outputs
    
def onnxruntime_inference(ort_session, input_ids, past=None, total_runs=100):    
    # Use contiguous array as input might improve performance.
    # You can check the results from performance test tool to see whether you need it.
    ort_inputs = {
        'input_ids':  numpy.ascontiguousarray(input_ids.cpu().numpy())
    }
    
    if past is not None:
        for i, past_i in enumerate(past):
            ort_inputs[f'past_{i}'] = numpy.ascontiguousarray(past[i].cpu().numpy())
            
    latency = []
    for _ in range(total_runs):
        start = time.time()
        ort_outputs = ort_session.run(None, ort_inputs)
        latency.append(time.time() - start)
        
    if total_runs > 1:
        print("OnnxRuntime Inference time = {} ms".format(format(sum(latency) * 1000 / len(latency), '.2f')))
    
    return ort_outputs

def inference(model, ort_session, input_ids, past=None, total_runs=100, verify_outputs=True):
    outputs = pytorch_inference(model, input_ids, past, total_runs)
    ort_outputs = onnxruntime_inference(ort_session, input_ids, past, total_runs)
    if verify_outputs:
        print('PyTorch and OnnxRuntime output 0 (last_state) are close:'.format(0), numpy.allclose(ort_outputs[0], outputs[0].cpu(), rtol=1e-05, atol=1e-04))

        if enable_past_input:
            for layer in range(model.config.n_layer):
                print('PyTorch and OnnxRuntime layer {} state (present_{}) are close:'.format(layer, layer), numpy.allclose(ort_outputs[1 + layer], outputs[1][layer].cpu(), rtol=1e-05, atol=1e-04))    

In [ ]:
import torch
import os

inputs = tokenizer.encode_plus("Here is an example input for GPT2 model", add_special_tokens=True, return_tensors='pt')
input_ids = inputs['input_ids']

# run without past so that we can know the shape of past from output.
outputs = model(input_ids=input_ids, past=None)

In [ ]:
num_layer = model.config.n_layer    
present_names = [f'present_{i}' for i in range(num_layer)]
output_names = ["last_state"] + present_names

input_names = ['input_ids']
dynamic_axes= {'input_ids': {0: 'batch_size', 1: 'seq_len'},
               #'token_type_ids' : {0: 'batch_size', 1: 'seq_len'},
               #'attention_mask' : {0: 'batch_size', 1: 'seq_len'},
               'last_state' : {0: 'batch_size', 1: 'seq_len'}
              }
for name in present_names:
        dynamic_axes[name] = {1: 'batch_size', 3: 'seq_len'}
        
if enable_past_input:
    past_names = [f'past_{i}' for i in range(num_layer)]
    input_names = ['input_ids'] + past_names  #+ ['token_type_ids', 'attention_mask']
    dummy_past = [torch.zeros(list(outputs[1][0].shape)) for _ in range(num_layer)]
    for name in past_names:
        dynamic_axes[name] = {1: 'batch_size', 3: 'seq_len'}
    export_inputs = (inputs['input_ids'], tuple(dummy_past)) #, inputs['token_type_ids'], inputs['attention_mask'])
else:
    export_inputs = (inputs['input_ids'])

export_model_path = os.path.join(output_dir, 'gpt2_past{}.onnx'.format(int(enable_past_input)))

torch.onnx.export(model,
                  args=export_inputs,
                  f=export_model_path,
                  input_names=input_names,
                  output_names=output_names,
                  dynamic_axes=dynamic_axes,
                  opset_version=11,
                  do_constant_folding = True,
                  verbose=False)

In [ ]:
def remove_past_outputs(export_model_path, output_model_path):
    from onnx import ModelProto
    from OnnxModel import OnnxModel

    model = ModelProto()
    with open(export_model_path, "rb") as f:
        model.ParseFromString(f.read())
    bert_model = OnnxModel(model)

    # remove past state outputs and only keep the first output.
    keep_output_names = [bert_model.model.graph.output[0].name]
    logger.info(f"Prune graph to keep the first output and drop past state outputs:{keep_output_names}")
    bert_model.prune_graph(keep_output_names)

    bert_model.save_model_to_file(output_model_path)
    
if enable_past_input:
    onnx_model_path = export_model_path
else:
    onnx_model_path = os.path.join(output_dir, 'gpt2_past{}_out1.onnx'.format(int(enable_past_input)))
    remove_past_outputs(export_model_path, onnx_model_path)

## Inference with ONNX Runtime

### OpenMP Environment Variable

OpenMP environment variables are very important for CPU inference of GPT2 model. It has large performance impact on GPT2 model so you might need set it carefully according to benchmark script.

Setting environment variables shall be done before importing onnxruntime. Otherwise, they might not take effect.

In [ ]:
import psutil

# You may change the settings in this cell according to Performance Test Tool result.
use_openmp = True

# ATTENTION: these environment variables must be set before importing onnxruntime.
if use_openmp:
    os.environ["OMP_NUM_THREADS"] = str(psutil.cpu_count(logical=True))
else:
    os.environ["OMP_NUM_THREADS"] = '1'

os.environ["OMP_WAIT_POLICY"] = 'ACTIVE'

In [ ]:
import onnxruntime
import numpy

# Print warning if user uses onnxruntime-gpu instead of onnxruntime package.
if 'CUDAExecutionProvider' in onnxruntime.get_available_providers():
    print("warning: onnxruntime-gpu is not built with OpenMP. You might try onnxruntime package to test CPU inference.")

sess_options = onnxruntime.SessionOptions()

# Optional: store the optimized graph and view it using Netron to verify that model is fully optimized.
# Note that this will increase session creation time, so it is for debugging only.
#sess_options.optimized_model_filepath = os.path.join(output_dir, "optimized_model_cpu.onnx")
   
if use_openmp:
    sess_options.intra_op_num_threads=1
else:
    sess_options.intra_op_num_threads=psutil.cpu_count(logical=True)

# Specify providers when you use onnxruntime-gpu for CPU inference.
session = onnxruntime.InferenceSession(onnx_model_path, sess_options, providers=['CPUExecutionProvider'])

# Compare PyTorch and OnnxRuntime inference performance and results
%time inference(model, session, input_ids, past=dummy_past if enable_past_input else None)

In [ ]:
import gc
del session
gc.collect()

In [ ]:
optimized_model = os.path.join(output_dir, 'gpt2_past{}_optimized.onnx'.format(int(enable_past_input)))

In [ ]:
bert_opt_script = os.path.join(bert_tools_dir, 'optimizer.py')

In [ ]:
# Local directory corresponding to https://github.com/microsoft/onnxruntime/tree/master/onnxruntime/python/tools/transformers/
%run $bert_opt_script --model_type gpt2 --input $onnx_model_path --output $optimized_model --opt_level 0

In [ ]:
session = onnxruntime.InferenceSession(optimized_model, sess_options, providers=['CPUExecutionProvider'])

%time inference(model, session, input_ids, past=dummy_past if enable_past_input else None, verify_outputs=False)

## Additional Info

Note that running Jupyter Notebook has slight impact on performance result since Jupyter Notebook is using system resources like CPU and memory etc. It is recommended to close Jupyter Notebook and other applications, then run the benchmark script in a console to get more accurate performance numbers.

[OnnxRuntime C API](https://github.com/microsoft/onnxruntime/blob/master/docs/C_API.md) could get slightly better performance than python API. If you use C API in inference, you can use OnnxRuntime_Perf_Test.exe built from source to measure performance instead.

Here is the machine configuration that generated the above results. The machine has GPU but not used in CPU inference.
You might get slower or faster result based on your hardware.

In [ ]:
machine_info_script = os.path.join(bert_tools_dir, 'MachineInfo.py')
%run $machine_info_script --silent